In [1]:
import torch
import pickle
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm


%run 'Dataset_and_preprocessing.ipynb'

%run 'model.ipynb'

print("Source Vocabulary Size:", len(src_vocab))
print("Target Vocabulary Size:", len(tgt_vocab))


# Set random seed for PyTorch CPU operations
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE).to(device)

# Initialize the attention layer
attention_layer = BahdanauAttention(units)

# Initialize the decoder

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE).to(device)

# Optimizer
optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=0.001)

# Custom Loss Function
def loss_function(real, pred):
    # Mask for non-zero tokens in the target
    mask = real.ne(0)
    loss = F.cross_entropy(pred, real, reduction='none')
    loss = loss * mask
    return loss.mean()

# Training Step Function
def train_step(inp, targ, enc_hidden):
    # Move data to the device
    inp, targ, enc_hidden = inp.to(device), targ.to(device), enc_hidden.to(device)

    loss = 0
    optimizer.zero_grad()

    current_batch_size = inp.size(0)
    enc_hidden = enc_hidden[:, :current_batch_size, :]

    enc_output, enc_hidden = encoder(inp, enc_hidden)
    dec_hidden = enc_hidden

    sos_token_index = tgt_vocab['<sos>']
    dec_input = torch.full((current_batch_size, 1), sos_token_index, dtype=torch.long, device=inp.device)

    for t in range(1, targ.size(1)):
        predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
        loss += loss_function(targ[:, t], predictions.squeeze(1))
        dec_input = targ[:, t].unsqueeze(1)

    batch_loss = loss / int(targ.size(1))
    batch_loss.backward()
    optimizer.step()

    return batch_loss.item()

# Training Loop
EPOCHS = 40
for epoch in tqdm(range(EPOCHS)):
    total_loss = 0

    for inp, targ in train_loader:
        current_batch_size = inp.size(0)
        # Initialize hidden state with the correct current batch size
        enc_hidden = encoder.initialize_hidden_state(current_batch_size).to(device)

        
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    if epoch % 4 == 0:
        print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(train_loader)}')

        
        
#save the model 
torch.save(encoder.state_dict(), 'encoder.pth')
torch.save(decoder.state_dict(), 'decoder.pth')

Batch shapes: torch.Size([64, 20]) torch.Size([64, 15])
2107
2131
Device: cpu
2107
2131
Source Vocabulary Size: 2107
Target Vocabulary Size: 2131


  2%|█                                           | 1/40 [01:17<50:41, 78.00s/it]

Epoch 1/40, Loss: 2.2938504954601857


 12%|█████▌                                      | 5/40 [05:51<40:25, 69.31s/it]

Epoch 5/40, Loss: 1.1056782730082249


...

 82%|███████████████████████████████████▍       | 33/40 [43:17<10:09, 87.03s/it]

Epoch 33/40, Loss: 0.01705871930623308


 92%|███████████████████████████████████████▊   | 37/40 [48:42<04:06, 82.17s/it]

Epoch 37/40, Loss: 0.016087393593439397


100%|███████████████████████████████████████████| 40/40 [52:44<00:00, 79.11s/it]
